# pandas 进阶修炼
<br>

本习题基于 `pandas` 版本 `1.1.3`，所有内容应当在 `Jupyter Notebook` 中执行以获得最佳效果。

不同版本之间写法可能会有少许不同，如若碰到此情况，你应该学会如何自行检索解决。

## 5 - 数据筛选与修改


<br>

现在，终于来到 `pandas` 数据分析中最**高频**的操作，没有之一————数据的`增删改查`

在后面的分组、聚合、透视、可视化等多个操作中，数据的筛选、修改操作也会不断出现。

因此在本章节，我整理了 40 余个常用操作

以习题的形式，带大家回顾利用 `pandas` 进行数据的增删改查。

需要注意的是，**本节中的大部分习题都有两个、三个或者更多的解法**，但我仅提供一个思路。

我提供的并非唯一也并非最优的方法，欢迎大家集思广益，加入 pandas 星球来提交你的解答并获得一定奖励。



## 初始化

<br>

该 `Notebook` 版本为**习题+答案版**


请执行下方单元格以初始化读取本节习题的答案

执行完毕后，你可以使用 `ans(1)` 来查看第 1 题的参考解答，其余习题类似。

注意：所有答案并非固定（唯一），我提供的答案仅供参考（并非推荐答案或者最优解）



In [ ]:
import sys
sys.path.append('../document/')
from inits import initialize, ans
res = initialize("../document/answer-5.txt")

## 加载数据

In [ ]:
import pandas as pd

# df = pd.read_html("https://olympics.com/tokyo-2020/olympic-games/zh/results/all-sports/medal-standings.htm")[0]
# 先打开文件来观察csv文件的内容，但是如果数据量较大的话，还是使用head，info，describe函数较好
df = pd.read_csv("东京奥运会奖牌数据.csv")  # 如果未联网可以使用这条命令加载数据

## 5-1 数据修改



### 1- <font color = '#FB8E00'>数据修改</font>｜列名

<br>

将原 `df` 列名 `Unnamed: 2`、`Unnamed: 3`、`Unnamed: 4` 修改为 `金牌数`、`银牌数`、`铜牌数`

In [ ]:
df.rename(columns = {
    'Unnamed: 2': '金牌数',
    'Unnamed: 3': '银牌数',
    'Unnamed: 4': '铜牌数'}, inplace = True)

### 2 - <font color = '#FB8E00'>数据修改</font>｜行索引

<br>

将第一列（排名）设置为索引

In [ ]:
df.set_index("排名", inplace = True)

### 3 - <font color = '#FB8E00'>数据修改</font>｜修改索引名

<br>

修改索引名为 金牌排名

In [ ]:
df.rename_axis("金牌排名", inplace = True)

### 4 - <font color = '#FB8E00'>数据修改</font>｜修改值

<br>

将 ROC（第一列第五行）修改为 俄奥委会

In [ ]:
df.iloc[4, 0] = '俄奥委会'

### 5 - <font color = '#FB8E00'>数据修改</font>｜替换值（单值）

<br>

将金牌数列的数字 `0` 替换为 `无`

In [ ]:
df['金牌数'].replace(0, '无', inplace = True)

### 6 - <font color = '#FB8E00'>数据修改</font>｜替换值（多值）



同时替换

- 将 `无` 替换为 缺失值
- 将 0 替换为 `None`

注意：缺失值的 Nan 该怎么生成？

In [ ]:
import numpy as np

df.replace(['无', 0], [np.nan, 'None'], inplace = True)

### 7 - 数据查看

查看各列数据类型

In [ ]:
df.dtypes

In [ ]:
numeric_features = df.select_dtypes(include = [np.number])

### 8 - <font color = '#FB8E00'>数据修改</font>｜修改类型

将 `金牌数` 列类型修改为 `int`

In [ ]:
# df['金牌数'].fillna('0') 将NULL值填充为0
df['金牌数'] = df['金牌数'].fillna('0').astype(int)

### 9 - <font color = '#1B85FF' >数据增加</font>｜新增列（固定值）

<br>

**重新加载数据** 并 **新增一列** 比赛地点，值为 `东京`


In [ ]:
df['比赛地点'] = '东京'

### 10 - <font color = '#1B85FF' >数据增加</font>｜新增列（计算值）

<br>

**新增一列** 金银牌总数列，值为该国家金银牌总数

In [ ]:
# 保证数据类型一样
df = df.replace('None', 0)
df['金银牌总数'] = df['金牌数'] + df['银牌数']

### 11 - <font color = '#1B85FF' >数据增加</font>｜新增列（比较值）

新增一列 最多奖牌数量 列，值为该国 `金、银、铜` 牌数量中最多的一个奖牌数量

例如美国银牌最多，则为41，中国为38

In [ ]:
# 纵向取每列的最大值
df[['金牌数', '银牌数', '铜牌数']].max()
df[['金牌数', '银牌数', '铜牌数']].max(0)
df[['金牌数', '银牌数', '铜牌数']].max(axis = 0)
df[['金牌数', '银牌数', '铜牌数']].max(axis = 'index')

# 横向取每行三列中的最大值
df[['金牌数', '银牌数', '铜牌数']].max(1)

In [ ]:
df['最多奖牌数量'] = df.bfill(1)[['金牌数', '银牌数', '铜牌数']].max(1)

### 12 - <font color = '#1B85FF' >数据增加</font>｜新增列（判断值）

新增一列 金牌大于30

如果一个国家的金牌数大于 30 则值为 是，反之为 否

In [ ]:
df['金牌数大于 30'] = np.where(df['金牌数'] > 30, '是', '否')

### 13 - <font color = '#1B85FF' >数据增加</font>｜增加多列

新增两列，分别是

- 金铜牌总数（金牌数+铜牌数）
- 银铜牌总数（银牌数+铜牌数）

In [ ]:
df = df.assign(
    金铜牌总数 = df.金牌数 + df.铜牌数,
    银铜牌总数 = df.银牌数 + df.铜牌数
)

### 14 - <font color = '#1B85FF' >数据增加</font>｜新增列（引用变量）

新增一列金牌占比，为各国金牌数除以总金牌数（gold_sum）

In [ ]:
gold_sum = df['金牌数'].sum()

In [ ]:
df.eval(f'金牌占比 = 金牌数/{gold_sum}', inplace = True)

### 15 - <font color = '#1B85FF' >数据增加</font>｜新增行（末尾追加）

<br>

在 df 末尾追加一行，内容为 0,1,2,3... 一直到 `df` 的列长度

In [ ]:
# pd.DataFrame(值, 列名)
df1 = pd.DataFrame([[i for i in range(len(df.columns))]], columns=df.columns)
# append默认方向为纵向
df_new = df.append(df1)

### 16 - <font color = '#1B85FF' >数据增加</font>｜新增行（指定位置）

<br>

在第 2 行新增一行数据，即美国和中国之间。

数据内容同上一题

In [ ]:
# 第一行数据
df1 = df.iloc[:1, :]
# 第二行至结尾的数据
df2 = df.iloc[1:, :]
df3 = pd.DataFrame([[i for i in range(len(df.columns))]], columns = df.columns)
df_new = pd.concat([df1, df3, df2], ignore_index = True)

### 17 - <font color ='#27BE49'>数据删除</font>｜删除行

<br>

删除 `df` 第一行

In [ ]:
# 删除index为1的行
# index从0开始
df.drop(1)

### 18 - <font color ='#27BE49'>数据删除</font>｜删除行（条件）

In [ ]:
# df.金牌数 < 20 判断每一行条件结果
# df[df.金牌数 < 20].index 找到符合条件每一行的索引
df = df.drop(df[df.金牌数 < 20].index)

### 19 - <font color ='#27BE49'>数据删除</font>｜删除列

<br>

删除刚刚新增的 比赛地点 列

### 20 - <font color ='#27BE49'>数据删除</font>｜删除列（按列号）

<br>

删除 `df` 的 `7、8、9、10` 列

## 5-2 数据筛选

以下所有答案要求返回的是一个 dataframe 而不是 Series 这样可以直接存储为 Excel 等格式的文件！

### 21 - 重新加载数据

In [ ]:
import pandas as pd

df = pd.read_html("https://olympics.com/tokyo-2020/olympic-games/zh/results/all-sports/medal-standings.htm")[0]

# df = pd.read_csv("东京奥运会奖牌数据.csv")  # 如果未联网可以使用这条命令加载数据

df.rename(columns={'Unnamed: 2':'金牌数',
                  'Unnamed: 3':'银牌数',
                  'Unnamed: 4':'铜牌数'},inplace=True)


### 22 - <font color = '#FB8E00'>筛选列</font>｜通过列号

提取第 1、2、3、4 列

In [ ]:
# 索引不算在正常列中
df.iloc[:, [0, 1, 2, 3]]

### 23 - <font color = '#FB8E00'>筛选列</font>｜通过列名

提取 `金牌数、银牌数、铜牌数` 三列

In [ ]:
df[['金牌数', '银牌数', '铜牌数']]

### 24 - <font color = '#FB8E00'>筛选列</font>｜条件（列号）

筛选全部 **奇数列**

In [ ]:
df.iloc[:, [i%2==1 for i in range(len(df.columns))]]

### 25 - <font color = '#FB8E00'>筛选列</font>｜条件（列名）

提取全部列名中以 “数” 结尾的列

In [ ]:
df.loc[:, df.columns.str.endswith('数')]

### 26 - <font color = '#FB8E00'>筛选列</font>｜组合（行号+列名）

提取倒数后三列的10-20行

In [ ]:
# 10:20 提取10-20行
# '总分': 提取总分到最后
df.loc[10:20, '总分':]

### 27 - <font color = '#1B85FF' >筛选行</font>｜通过行号

提取第 10 行

In [ ]:
df.loc[9:9]

In [ ]:
# 返回值类型为dataframe
df.iloc[[9]]
# 返回值类型为series
df.iloc[9]

### 28 -  <font color = '#1B85FF' >筛选行</font>｜通过行号（多行）

提取第 10 行之后的全部行

In [ ]:
df.loc[9:]

### 29 - <font color = '#1B85FF' >筛选行</font>｜固定间隔

提取 0-50 行，间隔为 3

In [ ]:
df[:50:3]

### 30 -  <font color = '#1B85FF' >筛选行</font>｜判断（大于）

提取 金牌数 大于 30 的行

In [ ]:
# df['金牌数'] > 30 ：'金牌数'这一列的情况
df[df['金牌数'] > 30]

### 31 -  <font color = '#1B85FF' >筛选行 </font>｜判断（等于）

提取 `金牌数` 等于 10 的行

In [ ]:
df[df['金牌数'] == 10]
df.loc[df['金牌数'] == 10]

### 32 - <font color = '#1B85FF' >筛选行</font>｜判断（不等于）

提取 `金牌数` 不等于 `10` 的行

In [ ]:
df[df['金牌数'] != 10]
df.loc[~(df['金牌数'] == 10)]

### 33 - <font color = '#1B85FF' >筛选行</font>｜条件（指定行号）

提取全部 **奇数行**


In [ ]:
# index取行，column取列
df[[i%2==1 for i in range(len(df.index))]]

### 34 - <font color = '#1B85FF' >筛选行</font>｜条件（指定值）


提取 中国、美国、英国、日本、巴西 五行数据

In [ ]:
df[df['国家奥委会'].isin(['中国', '美国', '英国', '日本', '巴西'])]
df.loc[df['国家奥委会'].isin(['中国', '美国', '英国', '日本', '巴西'])]

### 35 -  <font color = '#1B85FF' >筛选行</font>｜多条件

在上一题的条件下，新增一个条件：**金牌数小于30**

In [ ]:
df.loc[(df['金牌数'] < 30) & (df['国家奥委会'].isin(['中国', '美国', '英国', '日本', '巴西']))]

### 36 - <font color = '#1B85FF' >筛选行</font>｜条件（包含指定值）

提取 国家奥委会 列中，所有包含 `国`的行

In [ ]:
df[df.国家奥委会.str.contains('国')]

### 37 - 筛选某行某列

提取 `第 0 行第 2 列`

In [ ]:
df.iloc[0:1, [1]]

### 38 - 筛选多行多列


提取 第 0-2 行第 0-2 列


In [ ]:
df.iloc[0:2, [0,1]]

### 39 - <font color ='#27BE49'>筛选值</font>｜组合（行号+列号）

提取第 4 行，第 4 列的值

In [ ]:
df.iloc[3, 3]

###  40 - <font color ='#27BE49'>筛选值</font>｜组合（行号+列名）

提取行索引为 4 ，列名为 金牌数 的值


In [ ]:
df.at[4, '金牌数']
df.loc[4, '金牌数']

### 41 - <font color ='#27BE49'>筛选值</font>｜条件

提取 国家奥委会 为 中国 的金牌数

In [ ]:
df.loc[df['国家奥委会'] == '中国'].loc[1].at['金牌数']

### 42 - <font color ='#27BE49'>筛选值</font>｜query

使用 query 提取 金牌数 + 银牌数 大于 15 的国家

In [ ]:
df.query('金牌数+银牌数 > 15')

###  43 - <font color ='#27BE49'>筛选值</font>｜query（引用变量）

使用 query 提取 金牌数 大于 金牌均值的国家

In [ ]:
gold_mean = df['金牌数'].mean()

In [ ]:
df.query(f'金牌数 > {gold_mean}')